In [2]:
# import pickle
# # path = "data/Physics/0_uncopynode.pkl"
# # with open(path, 'rb') as f:
# #     data = pickle.load(f)
# # print(data.edge_index.max())
# # print(len(data.x))

In [41]:
# # print(data.edge_index.shape, data.train_mask.shape)
# # s = set()
# # for e in data.edge_index[1]:
# #     s.add(e.item())
# # print(len(s))
# # data.edge_index
# # data.train_mask

# # path = 'data/SBM/0_uncopynode.pkl'
# # with open(path, 'rb') as f:
# #     data = pickle.load(f)
# # print(data.y.min(),data.y.max())
# # print(data.edge_index[0].max())
# # print(data.edge_index[1].max())
# # print(len(data.x[0]))
# import torch
# x = torch.Tensor([0,0])
# print(x.to('cuda').grad)


None


In [11]:
import pickle
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
with open(path_sbm_train, 'rb') as f:
    sbm_train = pickle.load(f)
with open(path_sbm_val, 'rb') as f:
    sbm_val = pickle.load(f)
with open(path_sbm_test, 'rb') as f:
    sbm_test = pickle.load(f)
# sbm_train

In [1]:
# # for tmp in sbm_train:
# #     print(len(tmp.node_feat))
# # sbm_train[0].node_feat
# # edge_index = torch.tensor([[],[]])
# # print(edge_index)
# # print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# # x = torch.BoolTensor([0]*10)
# # x[1:3] = True
# # print(x)
# # print([1,2]*2)
# torch.nonzero(sbm_train[0].node_feat)

# torch.nonzero(sbm_train[0].W).T[:,torch.nonzero(sbm_train[0].W).T[0] == 20]
# for index in [  9,  13,  15,  19,  23,  24,  26,  31,  35,  41,  51,  52,  55,  58,
#           64,  65,  67,  72,  75,  76,  78,  80,  84,  86,  87,  89,  90,  92,
#           93,  94,  96,  99, 101, 102, 105, 111, 115]:
#     print(sbm_train[0].node_label[index])
# # s_train = 0
# # for tmp in sbm_train:
# #     s_train += len(tmp.node_feat)
# # s_val = 0
# # for tmp in sbm_val:
# #     s_val += len(tmp.node_feat)
# # s_test = 0
# # for tmp in sbm_test:
# #     s_test += len(tmp.node_feat)
# # print(s_train, s_val, s_test, s_train + s_val+s_test)


In [33]:
# from torch_geometric.data import Data
# K=1000
# NBTRAIN = len(sbm_train)
# NBVAL = len(sbm_val)
# NBTEST = len(sbm_test)

# import torch
# for k in range(K):
#     print(k)
#     idx_base=0
#     edge_index = torch.LongTensor([[],[]])
#     x = torch.tensor([])
#     y = torch.LongTensor([])
#     for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
#         sub_graph = sbm_train[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     # print(x.size(), edge_index, idx_base)
#     idx_train = idx_base
#     for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
#         sub_graph = sbm_val[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     # print(x.size(), edge_index, idx_base)
#     idx_val = idx_base
#     for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
#         sub_graph = sbm_test[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     # print(x.size(), edge_index, idx_base)
#     idx_test = idx_base

#     x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
#     train_mask = torch.BoolTensor([0]*idx_base)
#     train_mask[0:idx_train] = True
#     val_mask = torch.BoolTensor([0]*idx_base)
#     val_mask[idx_train:idx_val] = True
#     test_mask = torch.BoolTensor([0]*idx_base)
#     test_mask[idx_val:idx_test] = True
#     labels = y[train_mask]
#     weight = (labels.size(0) - torch.bincount(labels)).float() / labels.size(0)
#     with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
#         pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = val_mask, weight=weight), f)
#     # break


In [31]:
# from torch_geometric.data import Data
# K=1000
# NBTRAIN = len(sbm_train)
# NBVAL = len(sbm_val)
# NBTEST = len(sbm_test)

# import torch
# for k in range(K):
#     print(k)
#     idx_base=0
#     edge_index = torch.LongTensor([[],[]])
#     x = torch.tensor([])
#     y = torch.LongTensor([])
#     for idx_graph in range(k*int(NBTRAIN/K),min((k+1)*int(NBTRAIN/K),NBTRAIN)):
#         sub_graph = sbm_train[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     # print(x.size(), edge_index, idx_base)
#     idx_train = idx_base
#     x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
#     with open("data/SBM/{}_train.pkl".format(k), "wb") as f:
#         pickle.dump(Data(edge_index=edge_index, x = x, y = y), f)
#     # break


In [34]:
from models import Gat, Sage#, Gcn
from torch import optim
import pickle
import torch.nn.functional as F
import utils
import torch.nn as nn
import torch
import torch_geometric.nn as gnn


# model = Gat(6,6).to('cuda:0')
datas = []
for i in range(1000):
    path = f'data/SBM/{i}_uncopynode.pkl'
    with open(path, 'rb') as f:
        datas.append(pickle.load(f).to('cuda:3'))


In [13]:
label = datas[0].y[datas[0].train_mask]
weight = (label.size(0) - torch.bincount(label)).float() / label.size(0)


tensor([True, True, True, True, True, True], device='cuda:3')

In [36]:
torch.cuda.manual_seed(0)
class Gcn(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Gcn, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat,hdim)
        self.convs = nn.ModuleList([gnn.GCNConv(hdim, hdim) for _ in range(16)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x


class Sage(nn.Module):
    def __init__(self, nfeat, nclass):
        super(Sage, self).__init__()
        hdim = 162
        self.lnin = nn.Linear(nfeat, hdim)
        self.convs = nn.ModuleList(
            [gnn.SAGEConv(hdim, hdim) for _ in range(4)])
        self.lnout1 = nn.Linear(hdim, int(hdim/2), bias=True)
        self.lnout2 = nn.Linear(int(hdim/2), int(hdim/4), bias=True)
        self.lnout3 = nn.Linear(int(hdim/4), nclass, bias=True)
        return

    def forward(self, x, edge_index):
        x = self.lnin(x)
        for conv in self.convs:
            h_in = x
            x = F.relu(conv(x, edge_index))
            x = h_in + x
        x = F.relu(self.lnout1(x))
        x = F.relu(self.lnout2(x))
        x = self.lnout3(x)
        return x

model = Sage(6, 6).to('cuda:3')
model.train()

optimizer = optim.Adam(
    model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5)
for epoch in range(1000):
    losses = 0
    accus = 0
    test_accus = 0
    for iter, data in enumerate(datas):
        preds = model(data.x, data.edge_index)
        labels = data.y[data.train_mask]
        # weight = (labels.size(0) - torch.bincount(labels)).float() / labels.size(0)
        loss = F.cross_entropy(preds[data.train_mask], data.y[data.train_mask], weight=data.weight)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()
        accus += utils.accuracy(preds[data.train_mask],data.y[data.train_mask])
        test_accus += utils.accuracy(preds[data.test_mask],data.y[data.test_mask])
    print('Iter{}, loss:{},train_acc:{},test_acc:{}'.format(
        epoch+1, losses/(iter+1), accus/(iter+1),test_accus/(iter+1)))
    scheduler.step(losses/(iter+1))


Iter1, loss:1.7569509599208832,train_acc:0.23446633479751122,test_acc:0.2307111032555527
Iter2, loss:1.4716834785938262,train_acc:0.4199641755338665,test_acc:0.4238826719421657
Iter3, loss:1.3376061401367187,train_acc:0.4890791308066591,test_acc:0.490143959062184
Iter4, loss:1.2331563050746919,train_acc:0.5367983287047857,test_acc:0.5382275788495233
Iter5, loss:1.157402086675167,train_acc:0.5700316244116999,test_acc:0.5701767272504933
Iter6, loss:1.1100053730010986,train_acc:0.5902653561390814,test_acc:0.5906553444618663
Iter7, loss:1.0699517003893853,train_acc:0.6069379791084194,test_acc:0.6046941471913995
Iter8, loss:1.036080303311348,train_acc:0.62073185360214,test_acc:0.6170675352944555
Iter9, loss:1.013200150668621,train_acc:0.6300620083330006,test_acc:0.6259765211141258
Iter10, loss:0.987831940472126,train_acc:0.6400317575356204,test_acc:0.6374098798498222
Iter11, loss:0.9678427698016167,train_acc:0.6475960905315653,test_acc:0.6440764158478567
Iter12, loss:0.9520237830877304,trai

KeyboardInterrupt: 